# Pyscan use case with nD-gate control
## Erin Morissette, Brown University
### May 2024

This notebook demonstrates how pyscan is used to perform an n-D scan where n and D are linear combinations of voltages from top and bottom gates for a stacked graphene device.
 
With it pyscan users at Brown University make and take measurements from 5 separate lock-ins. <br />They also include experiment checks which ensure their parameters stay within safe operating conditions, protecting their instruments.

# Setup

In [ ]:
import sys

%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from time import sleep
from IPython import display
import pandas as pd
import matplotlib

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (7,5)
matplotlib.rcParams['font.size'] = 16
import pyscan as ps

from datetime import datetime
import pickle

from scipy.interpolate import interp1d

: 

## Instruments/Devices

In [ ]:
# Make a list of instruments
inst = ps.ItemAttribute()

inst.srs1 = ps.new_instrument(gpib_address=5)
inst.srs2 = ps.new_instrument(gpib_address=6)
inst.srs3 = ps.new_instrument(gpib_address=7)
inst.srs4 = ps.new_instrument(gpib_address=8)
inst.srs5 = ps.new_instrument(gpib_address=4)

inst.yoko17 = ps.new_instrument(gpib_address=17)
inst.yoko18 = ps.new_instrument(gpib_address=18)
# # inst.yoko19 = ps.new_instrument(gpib_address=19)

inst.keith24 = ps.new_instrument(gpib_address=24)

inst.ami = ps.new_instrument(visa_string = 'COM5', baud_rate = 115200)


: 

In [ ]:
# Assign drivers via devices
devices = ps.ItemAttribute()

devices.srs1 = ps.Stanford830(inst.srs1)
devices.srs2 = ps.Stanford830(inst.srs2)
devices.srs3 = ps.Stanford830(inst.srs3)
devices.srs4 = ps.Stanford830(inst.srs4)
devices.srs5 = ps.Stanford860(inst.srs5)

devices.top_gate = ps.YokogawaGS200(inst.yoko18)
devices.bottom_gate = ps.YokogawaGS200(inst.yoko17)
devices.si_gate = ps.Keithley2400(inst.keith24)


devices.magnet = ps.AmericanMagnetics430(inst.ami)

: 

## Functions

### Gate checks

In [ ]:
devices.top_gate.voltage_settings['range'] = [-10,10]
devices.bottom_gate.voltage_settings['range'] = [-10, 10]

: 

In [ ]:
def check_nD_nscan(runinfo, d_set):

    vt_range = [-10, 10]
    vb_range = [-10, 10]

    n_min = np.min(list(runinfo.loop0.scan_dict.values()))
    n_max = np.max(list(runinfo.loop0.scan_dict.values()))

    corners = [[n_min, d_set], [n_min, d_set], [n_max, d_set], [n_max, d_set]]

    vts = np.array([Vt_function(*corner) for corner in corners])
    vbs = np.array([Vb_function(*corner) for corner in corners])

    assert not np.any(vts < vt_range[0]) ,'Vt lower than range'
    assert not np.any(vts > vt_range[1]), 'Vt higher than range'
    assert not np.any(vbs < vb_range[0]), 'Vb lower than range'
    assert not np.any(vbs > vb_range[1]), 'Vb higher than range'

: 

In [ ]:
def check_nD_dscan(runinfo, n_set):

    vt_range = [-10, 10]
    vb_range = [-10, 10]

    d_min = np.min(list(runinfo.loop0.scan_dict.values()))
    d_max = np.max(list(runinfo.loop0.scan_dict.values()))

    corners = [[n_set, d_min], [n_set, d_min], [n_set, d_max], [n_set, d_max]]

    vts = np.array([Vt_function(*corner) for corner in corners])
    vbs = np.array([Vb_function(*corner) for corner in corners])

    assert not np.any(vts < vt_range[0]) ,'Vt lower than range'
    assert not np.any(vts > vt_range[1]), 'Vt higher than range'
    assert not np.any(vbs < vb_range[0]), 'Vb lower than range'
    assert not np.any(vbs > vb_range[1]), 'Vb higher than range'

: 

In [ ]:
def check_nD_map(runinfo):

    vt_range = [-10, 10]
    vb_range = [-10, 10]

    n_min = np.min(list(runinfo.loop0.scan_dict.values()))
    n_max = np.max(list(runinfo.loop0.scan_dict.values()))

    d_min = np.min(list(runinfo.loop1.scan_dict.values()))
    d_max = np.max(list(runinfo.loop1.scan_dict.values()))

    corners = [[n_min, d_min], [n_min, d_max], [n_max, d_min], [n_max, d_max]]

    vts = np.array([Vt_function(*corner) for corner in corners])
    vbs = np.array([Vb_function(*corner) for corner in corners])

    assert not np.any(vts < vt_range[0]) ,'Vt lower than range'
    assert not np.any(vts > vt_range[1]), 'Vt higher than range'
    assert not np.any(vbs < vb_range[0]), 'Vb lower than range'
    assert not np.any(vbs > vb_range[1]), 'Vb higher than range'

: 

### Measurement

In [ ]:
def measure_SRS(expt):
    devices = expt.devices
    runinfo = expt.runinfo
    
    d = ps.ItemAttribute()
    
    ix, itheta = devices.srs1.snap('x', 'theta')
    d.ix = ix
    d.itheta = itheta
    
    v1x, v1theta = devices.srs2.snap('x', 'theta')
    d.v1x = v1x
    d.v1theta = v1theta
    
    v2x, v2theta = devices.srs3.snap('x', 'theta')
    d.v2x = v2x
    d.v2theta = v2theta
    
    v3x, v3theta = devices.srs4.snap('x', 'theta')
    d.v3x = v3x
    d.v3theta = v3theta
    
    v4x, v4theta = devices.srs5.snap('x', 'theta')
    d.v4x = v4x
    d.v4theta = v4theta
    
    d.b0 = devices.magnet.field 
         
    return d

def end_function(expt):
    
    expt.devices.magnet.pause()
#     devices.top_gate.voltage = 0
#     devices.bottom_gate.voltage = 0
#     devices.srs5.sineout_voltage = 0
    

: 

### nD conversion (example)

In [ ]:
Ct = 0.00060
Cb = 0.00055
Vt0, Vb0 = 0.01, 0.01

n0 = -(Ct*Vt0_2 + Cb*Vb0)/1.602e-19

def n_function(Vt, Vb): # n, D in convenient units
    return ((Ct*Vt + Cb*Vb)/1.602e-19 + n0)/1e16

def D_function(Vt, Vb): # n, D in convenient units
    return (Ct*Vt -Cb*Vb)/(2*8.854e-12)/1e6

def Vt_function(n,D): # n, D in convenient units
    return (2*8.854e-12* (D*1e6) + 1.602e-19*((n*1e16) - n0))/2/Ct
           
def Vb_function(n,D): # n, D in convenient units
    return (-2*8.854e-12* (D*1e6) + 1.602e-19*((n*1e16) - n0))/2/Cb

def Vt_Vb(n, D):
    print('n = {}, D = {}, Vt = {}, Vb = {}'.format(n, D, round(Vt_function(n, D), 3), round(Vb_function(n, D),3)))

: 

# nD map

In [ ]:
if expt.runinfo.running == False:
    runinfo = ps.RunInfo()               

    runinfo.measure_function = measure_SRS
    runinfo.end_function = end_function

    # to save configuration 
    runinfo.new_runinfo = 'Enter additional experiment notes here'

    #### conversion from n/D to Vt/Vb ############
    ######################################################
    Ct = 0.00060
    Cb = 0.00055
    Vt0, Vb0 = 0.01, 0.01
    
    n0 = -(Ct*Vt0_2 + Cb*Vb0)/1.602e-19
    
    def n_function(Vt, Vb): # n, D in convenient units
        return ((Ct*Vt + Cb*Vb)/1.602e-19 + n0)/1e16
    
    def D_function(Vt, Vb): # n, D in convenient units
        return (Ct*Vt -Cb*Vb)/(2*8.854e-12)/1e6
    ######################################################


    def set_n(n):
        devices.top_gate.voltage = (Vt_function(n, expt.runinfo.D))
        devices.bottom_gate.voltage = (Vb_function(n, expt.runinfo.D))

    def set_D(D):
        expt.runinfo.D = D


    runinfo.loop0 = ps.FunctionScan(set_n, np.linspace(-1, 1, 200),  dt = 0.4)
    runinfo.loop1 = ps.FunctionScan(set_D, np.linspace(-200, 200, 150), dt = 1)


    check_nD_map(runinfo)


    expt = ps.PointByPointSweep(runinfo, devices)
    expt.start_thread()

    try:
        ps.live_plot1D(expt, data_name='v1x')
        plt.grid()
    except KeyboardInterrupt as p:
        expt.stop()
else:
    print('Previous experiment still running')

: 